In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.vision import *
from fastai.metrics import *
import glob
from shutil import copyfile, copy, move

In [81]:
path = Path('histopath')
temp = path/'temp'
train = path/'train'
test = path/'test'

In [9]:
files = list(path.rglob('*.*'))
len(files)

277524

In [10]:
np.random.seed(42)
np.random.shuffle(files)
files[:20]

[PosixPath('histopath/14153/0/14153_idx5_x1701_y2851_class0.png'),
 PosixPath('histopath/14304/0/14304_idx5_x1601_y1101_class0.png'),
 PosixPath('histopath/9346/0/9346_idx5_x601_y1201_class0.png'),
 PosixPath('histopath/12894/1/12894_idx5_x1101_y551_class1.png'),
 PosixPath('histopath/10288/0/10288_idx5_x1551_y251_class0.png'),
 PosixPath('histopath/15513/0/15513_idx5_x1151_y601_class0.png'),
 PosixPath('histopath/9178/0/9178_idx5_x801_y1751_class0.png'),
 PosixPath('histopath/15472/0/15472_idx5_x2251_y851_class0.png'),
 PosixPath('histopath/13462/0/13462_idx5_x2251_y1751_class0.png'),
 PosixPath('histopath/9126/1/9126_idx5_x1301_y1301_class1.png'),
 PosixPath('histopath/10288/0/10288_idx5_x801_y1401_class0.png'),
 PosixPath('histopath/10293/0/10293_idx5_x1901_y1201_class0.png'),
 PosixPath('histopath/9123/0/9123_idx5_x1801_y2201_class0.png'),
 PosixPath('histopath/9324/0/9324_idx5_x1101_y1401_class0.png'),
 PosixPath('histopath/13401/0/13401_idx5_x1101_y1301_class0.png'),
 PosixPath('

In [11]:
n_test = 0.2

In [12]:
n_files = len(files)
n_test = round(n_test * n_files)

print(n_files, n_test)

277524 55505


In [13]:
test_list = files[:55505]
len(test_list)

55505

In [62]:
#train_list = files[55505:]
#len(train_list+test_list)
len(train_list)
#train_list

222019

In [16]:
for i in range(len(test_list)):
    current_path = test_list[i]
    filename = current_path.stem + '.png'
    os.rename(current_path, test/filename)

In [40]:
for i in range(len(train_list)):
    current_path = train_list[i]
    filename = current_path.stem + '.png'
    label = str(current_path).split("_")[4].split(".")[0]
    if (label == "class0"):
        os.rename(current_path, train/'0'/filename)
    else:
        os.rename(current_path, train/'1'/filename)

In [99]:
for i in range(len(train_list)):
    current_path = train_list[i]
    filename = current_path.stem + '.png'
    label = str(current_path).split("_")[4].split(".")[0]
    #if (label == "class0"):
     #   os.rename(current_path, train/'0'/filename)
    os.rename(current_path, train/filename)

In [48]:
type(label)

str

In [59]:
names = []
class_labels = []

for n in train_list:
    name = n.stem
    names.append(name)
    class_label = name.split("_")[4][-1:]
    class_labels.append(class_label)

In [72]:
assert len(names)==len(class_labels)

In [73]:
d = {'imageID':names,'label':class_labels}

In [79]:
df = pd.DataFrame(d);df.head()

,imageID,label
0,8974_idx5_x2201_y1151_class0,0
1,9073_idx5_x2951_y851_class0,0
2,8951_idx5_x551_y701_class0,0
3,16085_idx5_x1301_y1301_class0,0
4,8975_idx5_x2001_y851_class1,1


In [84]:
df.to_csv(path/'data.csv')

In [ ]:
data = ImageDataBunch.from_folder(path, train, valid_pct = 0.2, ds_tfms = get_transforms, bs=32)

In [104]:
src = (ImageItemList.from_csv(".", 'data.csv', folder=train).random_split_by_pct().label_from_df('label'))

In [106]:
data = src.databunch(bs=64)

You can deactivate this warning by passing `no_check=True`.


/opt/anaconda3/lib/python3.7/site-packages/fastai/basic_data.py:226: UserWarning: There seems to be something wrong with your dataset, can't access any element of self.train_ds.
Tried: 76826,93375,21864,10745,109631...
  warn(warn_msg)


In [ ]:
data.c

In [107]:
data.show_batch(3, figsize=(12, 12))

FileNotFoundError: Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 138, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 138, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/anaconda3/lib/python3.7/site-packages/fastai/data_block.py", line 564, in __getitem__
    if self.item is None: x,y = self.x[idxs],self.y[idxs]
  File "/opt/anaconda3/lib/python3.7/site-packages/fastai/data_block.py", line 94, in __getitem__
    if isinstance(idxs, numbers.Integral): return self.get(idxs)
  File "/opt/anaconda3/lib/python3.7/site-packages/fastai/vision/data.py", line 273, in get
    res = self.open(fn)
  File "/opt/anaconda3/lib/python3.7/site-packages/fastai/vision/data.py", line 269, in open
    return open_image(fn, convert_mode=self.convert_mode)
  File "/opt/anac

In [82]:
df.to_csv(path/'data.csv')

In [41]:
datacsv = pd.read_csv(path/'data.csv')

In [83]:
datacsv.head()

NameError: name 'datacsv' is not defined

In [47]:
data = ImageItemList.from_df(path, df = datacsv, cols=files)

TypeError: from_df() got multiple values for argument 'df'

In [45]:
doc(ImageItemList.from_df)